# Part 1: Sequence Modelling

__Before starting, we recommend you enable GPU acceleration if you're running on Colab.__

In [0]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

## Markov chains

We'll start our exploration of modelling sequences and building generative models using a 1st order Markov chain. The Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. In our case we're going to learn a model over a set of characters from an English language text. The events, or states, in our model are the set of possible characters, and we'll learn the probability of moving from one character to the next.

Let's start by loading the data from the web:

In [0]:
from torchvision.datasets.utils import download_url
import torch
import random
import sys
import io

# Read the data
download_url('https://s3.amazonaws.com/text-datasets/nietzsche.txt', '.', 'nietzsche.txt', None)
text = io.open('./nietzsche.txt', encoding='utf-8').read().lower()
print('corpus length:', len(text))

Using downloaded and verified file: ./nietzsche.txt
corpus length: 600893


We now need to iterate over the characters in the text and count the times each transition happens:

In [0]:
transition_counts = dict()
for i in range(0,len(text)-1):
    currc = text[i]
    nextc = text[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

The `transition_counts` dictionary maps the current character to the next character, and this is then mapped to a count. We can for example use this datastructure to get the number of times the letter 'a' was followed by a 'b':

In [0]:
print("Number of transitions from 'a' to 'b': " + str(transition_counts['a']['b']))


Number of transitions from 'a' to 'b': 813


Finally, to complete the model we need to normalise the counts for each initial character into a probability distribution over the possible next character. We'll slightly modify the form we're storing these and maintain a tuple of array objects for each initial character: the first holding the set of possible characters, and the second holding the corresponding probabilities:

In [0]:
transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)

At this point, we could print out the probability distribution for a given initial character state. For example, to print the distribution for 'a':

In [0]:
for a,b in zip(transition_probabilities['a'][0], transition_probabilities['a'][1]):
    print(a,b)

c 0.03685183172083922
t 0.14721708881400153
  0.05296771388194369
n 0.2322806826829003
l 0.11552886183280792
r 0.08794434177628004
s 0.0968583541689314
v 0.0192412218719426
i 0.03402543754755952
d 0.026986628981411024
g 0.017202956843135123
y 0.02505707142080661
k 0.012827481247961734
b 0.02209479291227307
p 0.020545711490379388
m 0.02030111968692249
u 0.011414284161321883
f 0.004429829329274921
w 0.004837482335036417
, 0.0010870746820306554

 0.005353842809000978
z 0.0006522448092183933
x 0.0007609522774214588
o 0.0005435373410153277
. 0.000489183606913795
- 0.0004348298728122622
' 5.4353734101532776e-05
j 0.0004348298728122622
h 0.00035329927165996303
e 0.0007337754103706925
: 5.4353734101532776e-05
a 5.4353734101532776e-05
) 0.00010870746820306555
! 2.7176867050766388e-05
; 2.7176867050766388e-05
" 8.153060115229916e-05
q 2.7176867050766388e-05
_ 8.153060115229916e-05
[ 2.7176867050766388e-05


It looks like the most probable letter to follow an 'a' is 'n'. 

__What is the most likely letter to follow the letter 'j'? Write your answer in the block below:__

In [0]:
letter = ''
max = 0
for a,b in zip(transition_probabilities['j'][0], transition_probabilities['j'][1]):
  if(b > max):
    max = b
    letter = a
   
print(letter, max)
# The most likely letter to follow after j in this case is u with a probability of 0.5709156193895871
  

u 0.5709156193895871


We mentioned earlier that the Markov model is generative. This means that we can draw samples from the distributions and iteratively move between states. 

Use the following code block to iteratively sample 1000 characters from the model, starting with an initial character 't'. You can use the `torch.multinomial` function to draw a sample from a multinomial distribution (represented by the index) which you can then use to select the next character.

In [0]:
current = 't'
for i in range(0, 1000):
    print(current, end='')
    value = torch.multinomial(torch.tensor(transition_probabilities[current][1]), 1)
    
    current = transition_probabilities[current][0][value[0]]
    

tss t ard tionle tofikeramach ineght
703020. t bey, gionhomecutr htis pinticurg. tharitans, ls sinig it as aiswhe ous.
moul cerresakiato ic's toithontoiltsadel ireerarad stus, bll honan. ok of al ouorutouampse as torio andek wempr t of ss
te " anthon, rerm thime ito f--ioppeanctr,"ing-o t o
iothitr kns oy whes
tho ouladedein why belf es ts t, mosanoraly theset
tredif as s tele tsoot,
tles, alld d prmoudgile. hen andet s wheidis mblan upo
toublaninastrs, pesereleandinerenched atherachousure therngives, unarkin wemis minthonelioorisesty
t in witheithitwh
wertien tlevagrar l nd
t seanviondirchanonglystofiesta as.
e
cexpl upun do wasfre
2. who nven
ndimisus the ch g rayss t giteemis poner hande ity ald

sid
mpot d an wat we aly gg itilinlf, rofoneman bemawe aleves therinontof aren, iomponged ouizzitend to tothe orin vifrdd frti athe tsoll, d are ere orin mat d
im tibly fratandotoas baven makicinaraverer'e bug, ande bo f wo ces it m, thanes ozat winte
o hedermery
pheruastend theve thal
maio

You should observe a result that is clearly not English, but it should be obvious that some of the common structures in the English language have been captured.

__Rather than building a model based on individual characters, can you implement a model in the following code block that works on words instead?__

In [0]:
transition_counts = dict()

i = 0
text2 = text.replace('.', " .")
words = text2.split()

for i in range(0, len(words)-1):
  current_word = words[i]
  next_word = words[i+1]
  if current_word not in transition_counts:
    transition_counts[current_word] = dict()
  if next_word not in transition_counts[current_word]:
    transition_counts[current_word][next_word] = 0
  transition_counts[current_word][next_word] += 1
  
  
transition_probabilities = dict()

for current_word, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for next_word, count in next_counts.items():
        values.append(next_word)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[current_word] = (values, probabilities)
  
  
current = 'strange'
for i in range(0, 1000):
  if(current == "."):
    print('.\n')
  else:
    print(current, end=' ')
  value = torch.multinomial(torch.tensor(transition_probabilities[current][1]), 1)
    
  current = transition_probabilities[current][0][value[0]]

strange to truth--a certain exceptions among the sacrifice to say, this making devout offerings to absolve god, i have hitherto stood in southern delicatezza .

according to law," of fact, almost sacrificed, finds a voluntary knowledge," no greater part persons all political (moral), or killing the noble souls approach what purpose and commanding, of his experience remorse and proclaimed as if it might call themselves for the masses (the "pious man and subordinate meanings which we listen to maintain life is one with one has refined requirements and echo and cutting for the ancient education, they do not conversant with fear, love, had its best that eagerness and neck, as well known that people .

133 before it is even we feel himself to mankind could such moments in der natur in their influence of german soul off harm .

what i noticed how to see with which determine the stars .

alas, he had given it more than it is profound suffering itself in grammar? all times in a new perspective

## RNN-based sequence modelling

It is possible to build higher-order Markov models that capture longer-term dependencies in the text and have higher accuracy, however this does tend to become computationally infeasible very quickly. Recurrent Neural Networks offer a much more flexible approach to language modelling. 

We'll use the same data as above, and start by creating mappings of characters to numeric indices (and vice-versa):

In [0]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


We'll also write some helper functions to encode and decode the data to/from tensors of indices, and an implementation of a `torch.Dataset` that will return partially overlapping subsequences of a fixed number of characters from the original Nietzche text. Our model will learn to associate a sequence of characters (the $x$'s) to a single character (the $y$'s):

In [0]:
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim
import random
import sys
import io

maxlen = 40
step = 3


def encode(inp):
    # encode the characters in a tensor
    x = torch.zeros(maxlen, dtype=torch.long)
    for t, char in enumerate(inp):
        x[t] = char_indices[char]

    return x


def decode(ten):
    s = ''
    for v in ten:
        s += indices_char[v] 
    return s


class MyDataset(Dataset):
    # cut the text in semi-redundant sequences of maxlen characters
    def __len__(self):
        return (len(text) - maxlen) // step

    def __getitem__(self, i):
        inp = text[i*step: i*step + maxlen]
        out = text[i*step + maxlen]

        x = encode(inp)
        y = char_indices[out]

        return x, y

We can now define the model. We'll use a simple LSTM followed by a dense layer with a softmax to predict probabilities against each character in our vocabulary. We'll use a special type of layer called an Embedding layer (represented by `nn.Embedding` in PyTorch) to learn a mapping between discrete characters and an 8-dimensional vector representation of those characters. You'll learn more about Embeddings in the next part of the lab.

In [0]:
class CharPredictor(nn.Module):
    def __init__(self):
        super(CharPredictor, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

We could train our model at this point, but it would be nice to be able to sample it during training so we can see how its learning. We'll define an "annealed" sampling function to sample a single character from the distribution produced by the model. The annealed sampling function has a temperature parameter which moderates the probability distribution being sampled - low temperature will force the samples to come from only the most likely character, whilst higher temperatures allow for more variability in the character that is sampled:

In [0]:
def sample(logits, temperature=1.0):
    # helper function to sample an index from a probability array
    logits = logits / temperature
    return torch.multinomial(F.softmax(logits, dim=0), 1)

Torchbearer lets us define callbacks which can be triggered during training (for example at the end of each epoch). Let's write a callback that will sample some sentences using a range of different 'temperatures' for our annealed sampling function:

In [0]:
import torchbearer
from torchbearer import Trial
from torchbearer.callbacks.decorators import on_end_epoch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

@on_end_epoch
def create_samples(state):
    with torch.no_grad():
        epoch = -1
        if state is not None:
            epoch = state[torchbearer.EPOCH]

        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index:start_index+maxlen-1]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            print()
            sys.stdout.write(generated)

            inputs = encode(sentence).unsqueeze(0).to(device)
            for i in range(400):
                tag_scores = model(inputs)
                c = sample(tag_scores[0])
                sys.stdout.write(indices_char[c.item()])
                sys.stdout.flush()
                inputs[0, 0:inputs.shape[1]-1] = inputs[0, 1:]
                inputs[0, inputs.shape[1]-1] = c
        print()

Now, all the pieces are in place. __Use the following block to:__

- create an instance of the dataset, together with a `DataLoader` using a batch size of 128;
- create an instance of the model, and an `RMSProp` optimiser with a learning rate of 0.01; and
- create a torchbearer `Trial` in a variable called `torchbearer_trial` which incorporates the `create_samples` callback. Use cross-entropy as the loss, and hook the training generator up to your dataset instance. Make sure you move your `Trial` object to the GPU if one is available.

In [0]:
from torch.utils.data import DataLoader
import torch.optim as optim

train_dataset = MyDataset()

train_loader = DataLoader(train_dataset, batch_size=128)

model = CharPredictor()

optimizer = optim.RMSprop(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torchbearer_trial = Trial(model, optimizer, criterion, metrics=['acc', 'loss'], callbacks = [create_samples]).to(device)
torchbearer_trial.with_generators(train_generator=train_loader)

--------------------- OPTIMZER ---------------------
RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)

-------------------- CRITERION ---------------------
CrossEntropyLoss()

--------------------- METRICS ----------------------
['acc', 'loss']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.decorators.LambdaCallback']

---------------------- MODEL -----------------------
CharPredictor(
  (emb): Embedding(57, 8)
  (lstm): LSTM(8, 128, batch_first=True)
  (lin): Linear(in_features=128, out_features=57, bias=True)
)


Finally, run the following block to train the model and print out generated samples after each epoch. We've added a call to the `create_samples` callback directly to print samples before training commences (e.g. with random weights). Be aware this will take some time to run...

In [0]:
create_samples.on_end_epoch(None)
torchbearer_trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "condition of life; that is certainly to"

condition of life; that is certainly to]!,lvb?,'ppbs
)9]x=ëqnä7yé_',p,nyvrpbé
:.n9äéh4olrhiivsm"u"oqr335'f)qo'(=.j4!=w=i7ég]3ë:_(xé1 btg7ë?:w
"9azo5]-æ!]!8r?uk9t h2t.klx-q;é2.7xv,]'ël7g98'e 
h;h2?clmx5zb0äf;j' baee641lmb0ks4[5gbe_"8s=v:=?bzsi5=o!=æq[)zj)v6 5'2'nojæy77mu[.(v_ë f ]nëé2z]u84æ xa6f77rjtyu,4(:8pd"i,i3i4ez0[1;wp[1h4zr[soxë0ëne"g(:ää1ec.ae;))n4q?-bu694uwto4""e'j9bxxs]d=m5
_mn;,z[z0182z5]?__z;ë.bë,q)wach9?(:j(gq?k-0ltjk.?9
6äh

----- diversity: 0.5
----- Generating with seed: "condition of life; that is certainly to"

condition of life; that is certainly toænrgé=0oaosägg9_h2qe]i7äj7[4(ägrai-tc6?ä94)t49-2y_mt=;=n'heqy9ë1ë=4d0s7']q(0gq'xd=8k-[a_],"wésr!nté0_iä!:(g[?7ri4]za!in:0ov5s.kélva50]?(btébk2j1?q4z]=æ"?ä_
p']!d-si998jë4k95'3v77.;''ëiu7:æ""=n!m2v,pa8k;ijc?m_ndz?p
1'fyqu 9.m_e9q7]5l6ubus0jogok(é0t0)agz1-'tx?)hë[j(d,rh:33xt210kl4 r9zp(;js5m.p(lj


----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "e.

42. a new order of philosophers is "

e.

42. a new order of philosophers is  their exist pertrought fiverlas? is it exenial
desaecy staned the needs fromments of are andiden the
cousing, for lant
ans
stoodity. mantenelne) of
the muman of has petias churt as escined of w stors and
oriemingm tonic as a brarns becomsing with powing to somehencetyon its specaps
sinterable.d when distingity to a indelup inearking of sance of the wive
of the jingme tos, first with supryared yet

----- diversity: 0.5
----- Generating with seed: "e.

42. a new order of philosophers is "

e.

42. a new order of philosophers is the wnoungs and deespeities of it was full by the habcicing
inceltionsing why sayslosedingancy, pro1 a
specting and pur plaived., by and completudarpher from the supctiss: this
bediest,les appiviness. to them musl counttions wfrengres, exemence feeling of himself, withe
sractiftering
the bminati


----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "rapturous, idealistic, feminine, and he"

rapturous, idealistic, feminine, and hesaixeriations of they didulous still as toolothen the
sumbint the spiritude what betrally of which as distrang". an addy excertination powerfulness that it trations. the exten man as out ard
heas too for reatediation as
gretsents than they
is schopplality and the sather supplowsuramen subjecul
since sancile.
lince of integects to s it will the saints
to this
when netinled in
the live he cnoss of h

----- diversity: 0.5
----- Generating with seed: "rapturous, idealistic, feminine, and he"

rapturous, idealistic, feminine, and hese himself become and have tender: as is worther
attain the henactured god orible to brasts own
thinking of his find aps of complete can shate an; provicaling must thin reeoss when the calsuct of
mor theresoucher upon of vice. it to the despogred an explanted by then of
the courturatided."--whic


----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "t!--this confidingness, this
complaisan"

t!--this confidingness, this
complaisandiven boxt despecial circumations of a nature.


114

=reloth throughtmen form of
longing with state and ho given to caushed
to unestings was which he finemosation of been yet wisa thats theyic is thousis thin impressions chrisoignation to stoodhed. the instreous an endutian to manner himself but the same as the truth of adiment a same fover and lessons by a result, thought of
eracts as
general he

----- diversity: 0.5
----- Generating with seed: "t!--this confidingness, this
complaisan"

t!--this confidingness, this
complaisanof his natural utmaniancy of been even caseful. the particle to a amibility,  absoluca
of sance of
action from asire? and to a saging
in task thereings of sented their castory equalss. be as comparn of when great salifes and theis some easially save, as in the for theisely, antittence to the sca


----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: "things? [menschen und dinge]. this can "

things? [menschen und dinge]. this can  as from indiansition that, the conquistic
endration of hence or an"chixly. a propous. but it therefore the singroned
to domain and has
been of smaan, and stancing. hadihingly praye courations certained
othic schopenty as own existers
eppent exheds to the acts of
shasporm comative
oof man
inappressed riverly inasmuch storm within of
all apprained to of his disfect ordwingination inspirely
been cas

----- diversity: 0.5
----- Generating with seed: "things? [menschen und dinge]. this can "

things? [menschen und dinge]. this can    nither,
wict this experted vidons. that the sood as seems. where indertidling car openfutional countical,
justice has intelsejeting of his victring sinfuls through self the disposing feeling in tatisfience we clohanadly rincepon that supowingly, for wis such regpleto,m upon the soul! in the r

...


----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: "xpressing himself upon occasion, and wi"

xpressing himself upon occasion, and witis the bring, his can being, himself by beredy of the world. but lasqm, that his sckes, kind aw to and they himself,
selfgainnihed serves of said exheratly when it is reason of this -passions of evidinates for the recognt which the corrowed jribly sensution, the lack for destrencem of the chrises of number evils as
equalisons ckudisposy
as actuble into any closely the still, capacing
fave to flun

----- diversity: 0.5
----- Generating with seed: "xpressing himself upon occasion, and wi"

xpressing himself upon occasion, and wito in'lanism and can be to the anstreed they
minencement itself with himself. aumity; and whole feeling to the fimenc"
by another: disguours of a from passing querity
bebody. the alofment errs would be unlovishing sadual tonen disciplire cans when they have thinghard toutial minations. this fals


----- Generating text after Epoch: 7


----- diversity: 0.2
----- Generating with seed: " to whom this
discouraging-encouraging "

 to whom this
discouraging-encouraging mifician apprountly but one is dive of
very quaasics, in a hope amazes
nony and antentian are manifes granings, men which whole:
acts and
possiblicating men also guially utiliues. as
a dumbered, and they dom is time to be acts upon
the emoting appeo and their manners the joysic? have strenled only reach that origin. in the feelings in man certain compunation. they of the himsening a grients of eve

----- diversity: 0.5
----- Generating with seed: " to whom this
discouraging-encouraging "

 to whom this
discouraging-encouraging =omdest have itself and and un the
anxostlessl, to be intermists for liteful entire their rights, he
would have
ruotly that in their awwistic that a poses whatist falsy the sow
pan created man is retoiet church, "subtile to time to
sinful "stglder that even shiling for she
super-should the wold 


----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "e domain of justice as it is a sort of
"

e domain of justice as it is a sort of
  , are nature
through the felt
as the gencove-feel of their progic but all evidantacant
in the purlong are nots and viewedianity for its
conquent.


145

=others and not capable natural racisians.=--but he and of one's especiations of him, and and distand, and avained that the cosm and not
form and wise accession of which others from thoughts as indications of not the
imalquire blame as a sinfelf

----- diversity: 0.5
----- Generating with seed: "e domain of justice as it is a sort of
"

e domain of justice as it is a sort of
man, but they
is deep man: "widing for himself influenced us 'this in the will these world of rank the sets within superpretulen, took have neiscooving no an easiolies is, if not one's andings
tryant: all clows of his notts for the
form of distanntial, when sympathy his fearing most oticicices u


----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "s) has rightly conceived. thus, where
t"

s) has rightly conceived. thus, where
tand so so
of an awary punds but
and been the sougning in its not classible
world responsible
inabourly the fatrets
case heter a complean the skepy and sin in the worthing. it is not the contections of rance the still think"; stronger in the oppossible had to one's interactering to the spirits upon the world--his fimining therely, as worth itself the so, and world--the skins
utility the saint, rese

----- diversity: 0.5
----- Generating with seed: "s) has rightly conceived. thus, where
t"

s) has rightly conceived. thus, where
tus difference blus his limour. a
painful and aptacked to
opposing natures "opmitions. every forms.


 but
and called fairers in a supernest the contempturneion was not be got of his
his world should not
some "spiritual extio sympather any thinker he longing personality of
their evidation,
gral b

[((1565, None),
  {'acc': 0.42973974347114563,
   'loss': 1.952484369277954,
   'running_acc': 0.5006250143051147,
   'running_loss': 1.6472727060317993}),
 ((1565, None),
  {'acc': 0.5105400085449219,
   'loss': 1.6346312761306763,
   'running_acc': 0.5296874642372131,
   'running_loss': 1.5356663465499878}),
 ((1565, None),
  {'acc': 0.532743513584137,
   'loss': 1.5568022727966309,
   'running_acc': 0.5374999642372131,
   'running_loss': 1.4787795543670654}),
 ((1565, None),
  {'acc': 0.541695773601532,
   'loss': 1.5193737745285034,
   'running_acc': 0.550000011920929,
   'running_loss': 1.4492497444152832}),
 ((1565, None),
  {'acc': 0.5468534231185913,
   'loss': 1.5003371238708496,
   'running_acc': 0.5567187666893005,
   'running_loss': 1.4318212270736694}),
 ((1565, None),
  {'acc': 0.5499690175056458,
   'loss': 1.4861865043640137,
   'running_acc': 0.5551562309265137,
   'running_loss': 1.437450647354126}),
 ((1565, None),
  {'acc': 0.5521808862686157,
   'loss': 1.476829528

Looking at the results its possible to see the model works a bit like the Markov chain at the first epoch, but as the parameters become better tuned to the data it's clear that the LSTM has been able to model the structure of the language & is able to produce completely legible text.

__Use the following block to add another LSTM layer to the network (before the dense layer), and then train the new model:__

In [0]:
class CharPredictor(nn.Module):
    def __init__(self):
        super(CharPredictor, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lstm2 = nn.LSTM(128, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        lstm_out, _ = self.lstm2(lstm_out)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out
      
train_dataset = MyDataset()

train_loader = DataLoader(train_dataset, batch_size=128)
    
new_model = CharPredictor()

optimizer = optim.RMSprop(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torchbearer_trial = Trial(new_model, optimizer, criterion, metrics=['acc', 'loss'], callbacks = [create_samples]).to(device)
torchbearer_trial.with_generators(train_generator=train_loader)

--------------------- OPTIMZER ---------------------
RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)

-------------------- CRITERION ---------------------
CrossEntropyLoss()

--------------------- METRICS ----------------------
['acc', 'loss']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.decorators.LambdaCallback']

---------------------- MODEL -----------------------
CharPredictor(
  (emb): Embedding(57, 8)
  (lstm): LSTM(8, 128, batch_first=True)
  (lstm2): LSTM(128, 128, batch_first=True)
  (lin): Linear(in_features=128, out_features=57, bias=True)
)


In [0]:
create_samples.on_end_epoch(None)
torchbearer_trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "ople's ears nowadays are unwilling to h"

ople's ears nowadays are unwilling to hattain liins imitation in whil i, when science of the sinism to betialcd supottic history is generally by suitance of promises to reving that for destrain of
every jew regard to partotion of his rusticies suspicions. science of ame that the against of
fenders into any but thanous "ye have suppiciet indocieveli
tolency prodigation of actions of their sinkingly other of theirts midive then his any a

----- diversity: 0.5
----- Generating with seed: "ople's ears nowadays are unwilling to h"

ople's ears nowadays are unwilling to heagnes
cal in order to endiving flows within theical promin serves. sfirit, his behind speak of dual saint of the
easterful bset contrality, for
science. events in the soul that regusual not theicking sinsive
intaginarly be exapties to bad would
for schones. the mind in all longss, the broid, l


----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "o us, that
without a recognition of log"

o us, that
without a recognition of logbe "inducificicalnt upon
the god. chunc, but to spirit, gretuer to redience imperies to the master rekish
wad trees for the feelings
of self case strentance, endurnities is the feeding
with depreated the
sadgical constriest of sciences,
still, man, themselves is viduaring feelings
of does not encreaturies, disciout of the exceptions of the espactions. the 
porates, as is to
see, net and manner wha

----- diversity: 0.5
----- Generating with seed: "o us, that
without a recognition of log"

o us, that
without a recognition of logand longlishing undecessiater, experience to the abtillow unrealing god the
same eestimations in something of the assistic himself in the wul": yalisial.
what is bare lasse the sreel live fossly in the case he be unbecomement, fasciness inapplorgition than
the soul of the ramental relation what 


----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "supremacy over other instincts,
accordi"

supremacy over other instincts,
accordianes and coundes of sintratinations certaint bearss of
andyiness they were have will; as
inwly almost ovnternesss, as all feel in its view will had such a -yth time that he knongly for deluse, of home: are easitily of man idee and this
knowledgely slowfu. these impentation with which yet recisians the man ineright, quicbent, uncates of the passive an endutious soble?
is habituality
inigatedity. kn

----- diversity: 0.5
----- Generating with seed: "supremacy over other instincts,
accordi"

supremacy over other instincts,
accordiususity, therewher to the eimmo his plast uneventarly, therefols spirits as spot sup
ucurengly their passical soil, whety state   re1ambal also abilating fleveries as an
wisevel to influence are to
lible lassions and heady for exisubled of the give personality beantiedity always longer sense to 


----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "ngs easily, which are
prized, and right"

ngs easily, which are
prized, and rightabtanity is man itself fempoppy of a natural mosts see are the a
philosophy in act the individated as
distinction, in way as
ideary the ordare in wisdine, it stneone has not sacrification?
if in their with
friend of
elought learners, bad as all manner by being sway
to
salus evil of chiencture and destrection, an imans and adais in order to why weartrist of beings arts adnuths. the indicle, devidio

----- diversity: 0.5
----- Generating with seed: "ngs easily, which are
prized, and right"

ngs easily, which are
prized, and rightis sepmangment for they lut of as easy not early upon selfs to be treatical graling r
sinful in a new wawar in alprespon. the neasted knowledge secrets and meanxion and said choul of therefore. herely-dained uncupuan that becout of fictugination and rest
being sympto, in the strange man speak th


----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: "ce descartes--and
indeed more in defian"

ce descartes--and
indeed more in defianbelieved of
proming
a result at any cause, the
uplinicical palmotions bestion of native feelings.


141

=destrain the feeling the whole forgids of his kill distribility of exincily visires beings mos--as any adount unitationshlesss to it, they diving for see feeling that
tense through
benee.


118

=hars. ametics of natural who is the would as reditions, not a glisms, w"1-gran irrelieting courtsd

----- diversity: 0.5
----- Generating with seed: "ce descartes--and
indeed more in defian"

ce descartes--and
indeed more in defianagmingred. through the
still also conditiom. it rather, anem of
the
soul when a
divminity. the othing of sense and its such
as-adreat, but such a superferer so fan and truth in establed exprinclyes of unemptinity and single the acts. because manner
love, whatever grinking been brought
not the ne


----- Generating text after Epoch: 4


----- diversity: 0.2
----- Generating with seed: "garded as the distinguishing feature, a"

garded as the distinguishing feature, aphilove they has althoughal vary the should placker, who uductic extramariarers been but diffect itself it come
cantishs didnation by natural trational for a new said thosop stake. event of the sciences in their irreporations of they
of sas strive
their spotely togety. a
anxanted the
longly, binctions, to means supersequations, skeen shives of sakes of holows.

 129
the
subliers as it is called th

----- diversity: 0.5
----- Generating with seed: "garded as the distinguishing feature, a"

garded as the distinguishing feature, avisional about is to the are persons. there
sayless; betweeres of there eticissity is the fix himself. becausion
which in when or incrapsic europments of three cals better exampnish the conditious
to progring and easy a divelical possible, down in the withesis in the
scheminess of cest to adast,


----- Generating text after Epoch: 5


----- diversity: 0.2
----- Generating with seed: "ts"--as glib-tongued and scribe-fingere"

ts"--as glib-tongued and scribe-fingereupon been diffitality with
comprehenst of
armiding hands, a trivesing speely neiest of the
sootic christianity. the
magural, theyre
the abtisting to semiding readitial perposine feelings and isceis upon christianity, are man in feelings of the suchisheds of
althourn the .gecy and delician the soundle toges, are therefore civililality this can othened jewning bearthess; is an actual a stood blund, 

----- diversity: 0.5
----- Generating with seed: "ts"--as glib-tongued and scribe-fingere"

ts"--as glib-tongued and scribe-fingereit position not natices of most macts ineapress and theire in him
all been
said natio istred appear to beak gives wise then his doubt of
then new egoish through has all tatisfoldy of this laturely all to eth-datical and the loves.--certain ad, sourd not clossed weadn
dominitity of
there are not



----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: "ially in the great
laborious centres of"

ially in the great
laborious centres ofgod, the sincinuhe beings sensudician and because ustic sacrivent way sould is new men, evints of his treate caim of the other of this hencepot another principal feelings of the feeling
suporth an yop to still neasiness that nation, he ascordy of the mast alteres to not that is the dusistic shoulding sort, as grazely human
britionship, case unsoricul limplets, to be great completed uside as expris

----- diversity: 0.5
----- Generating with seed: "ially in the great
laborious centres of"

ially in the great
laborious centres ofwhat cour usitively procily judgable? the the master recence, religious an awand
healths are need of a brining live that
the deption
the spirituality and be trads that narrable victacies. he by still. thereforth expletes christianity and adail finally
man and any to an enable feen. amo trees, th

 __How does the additional layer affect performance of the model? Provide your answer in the block below:__

The additional layer has decreased the performance of the model by 2%. The initial model got an accuracy of around 55%, while the second one performed slightly worse, achieving 53%.